### 종합실습3
##### Requests 혹은 Selenium 모듈을 이용해 네이버 이미지 검색에서 아이유를 검색하고, 검색결과 이미지들을 PC에 저장해 보세요.

### 1. Requests로 크롤링하기

In [ ]:
import json
import requests
import urllib.parse

#SSL Verify 경고 끄기
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

#아래에 코드 작성
search_query = "아이유"
src = requests.get("https://search.naver.com/search.naver?where=image&sm=tab_jum&query="+search_query, verify = False)
json_start_str = "controller.start("
json_end_str = "); keep"
json_start_no = src.text.find(json_start_str) + len(json_start_str)
json_str = src.text[json_start_no:]
json_end_no = json_str.find(json_end_str)
json_str = src.text[json_start_no:json_start_no + json_end_no]

img_list = json.loads(json_str)

i = 0
for img in img_list:
    print("{}번째 이미지 다운로드 중".format(i), end="\r")
    if img["type"] == "image":
        i += 1
        this_img_url = urllib.parse.unquote(img["originalUrl"])
        this_image = requests.get(this_img_url, verify=False)
        try:
            this_ext = this_image.headers["Content-Disposition"].split(".")[-1]
        except:
            this_ext = "jpg"
        open("../outputs/images/naver/"+search_query+"_img"+str(i)+"."+this_ext, "wb").write(this_image.content)
print("{} 이미지 다운로드 완료".format(search_query), end="\r")

### 2. Selenium으로 크롤링하기

In [ ]:
import requests
import urllib.parse
from selenium import webdriver
from bs4 import BeautifulSoup

#SSL Verify 경고 끄기
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

#아래에 코드 작성
search_query = "아이유"
url = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query="+search_query
                   
driver = webdriver.Chrome("chromedriver.exe")
driver.get(url)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
images = soup.select("._listImage")
for i, img in enumerate(images):
    this_img_url_list = img.get("src").split("src=")
    if len(this_img_url_list) > 1:
        this_img_url = urllib.parse.unquote(this_img_url_list[1])
    else:
        this_img_url = urllib.parse.unquote(img.get("data-lazy-src"))
    #이미지 콘텐츠 다운로드는 requests 모듈을 써야 함
    this_image = requests.get(this_img_url, verify=False)
    try:
        this_ext = this_image.headers["Content-Disposition"].split(".")[-1]
    except:
        this_ext = "jpg"
    open("../outputs/images/naver/"+search_query+"_img"+str(i)+"."+this_ext, "wb").write(this_image.content)
print("{} 이미지 다운로드 완료".format(search_query))